In [1]:
import pandas as pd
import sqlalchemy as sa
import re
from sqlalchemy import create_engine, text
import numpy as np
import mysql.connector
from mysql.connector import Error

**LECTURA DE ARCHIVO**

In [2]:

# Patrón regex para el formato Combined Log de Apache
patron_regex = r'(\S+) (\S+) (\S+) \[(.*?)\] "([^"]+)" (\d{3}) (\S+) "([^"]*)" "([^"]*)"'

# Función para parsear el archivo de log
def parsear_archivo_log(ruta_archivo):
    datos = []
    with open(ruta_archivo, 'r', encoding='latin1') as archivo:
        for linea in archivo:
            coincidencia = re.match(patron_regex, linea)
            if coincidencia:
                datos.append(coincidencia.groups())
            else:
                print(f"Omitiendo línea malformada: {linea.strip()}")

    columnas = ['direccion_ip', 'identd', 'usuario', 'timestamp', 'request', 
                'codigo_estado', 'bytes_enviados', 'referer', 'agente_usuario']
    df = pd.DataFrame(datos, columns=columnas)
    
    # Separar el campo request en método, ruta y protocolo
    df[['metodo', 'ruta', 'protocolo']] = df['request'].str.split(' ', expand=True, n=2)
    df = df.drop(['request', 'identd', 'usuario', 'protocolo'], axis=1)
    
    # Eliminar la zona horaria (-0300) del timestamp
    df['timestamp'] = df['timestamp'].str.replace(" -0300", "", regex=False)
    #df['timestamp'] = df['timestamp'].str.replace(r'\s-0300$', '', regex=True)

    df['ruta'] = df['ruta'].str[:100]

    
    return df
# Uso
ruta_archivo = 'datos/access_ssl_20230404.log'  # Reemplaza con tu ruta real
df = parsear_archivo_log(ruta_archivo)


Omitiendo línea malformada: 20.100.172.53 - - [29/Jan/2023:09:56:13 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 20.208.136.113 - - [29/Jan/2023:21:59:09 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 51.11.244.26 - - [30/Jan/2023:09:29:23 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 20.7.152.157 - - [01/Feb/2023:16:15:10 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 20.7.152.157 - - [01/Feb/2023:16:15:34 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 20.7.152.157 - - [01/Feb/2023:16:26:48 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 20.7.152.157 - - [01/Feb/2023:18:19:35 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 20.7.152.157 - - [01/Feb/2023:18:29:27 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 20.7.152.157 - - [01/Feb/2023:19:07:34 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 93.90.200.43 - - [02/Feb/2023:18:56:09 -0300] "" 400 0 "-" "-"
Omitiendo línea malformada: 93.90.200.43 - - [02/Feb/2023:18:56:15 -0300] "" 400 0 "-" 

In [ ]:
def parsear_timestamp(timestamp_str):
    """
    Convierte timestamp de formato '29/Jan/2023:03:50:28' 
    a formato estándar '29-01-2023 03:50:28'
    """
    # Mapeo de meses en inglés a números
    meses = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
        'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
        'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }
    
    # Usar regex para extraer componentes: 29/Jan/2023:03:50:28
    pattern = r'(\d{1,2})/(\w{3})/(\d{4}):(\d{2}):(\d{2}):(\d{2})'
    match = re.match(pattern, timestamp_str)
    
    if match:
        dia, mes_str, anio, hora, minuto, segundo = match.groups()
        mes_num = meses.get(mes_str, '01')  # Default a enero si no encuentra el mes
        
        # Formato estándar: YYYY-MM-DD HH:MM:SS
        return f"{anio}-{mes_num}-{dia.zfill(2)} {hora}:{minuto}:{segundo}"
    else:
        return None
    
# Crear tablas de dimensiones y tabla de hechos
def crear_cubo_datos(df):
    df1 = df.copy()
    # Convertir timestamp a componentes
    df1['timestamp'] = df1['timestamp'].apply(parsear_timestamp)
    df1['timestamp'] = pd.to_datetime(df1['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    # Crear columna fecha (solo día completo)
    df1['fecha'] = df1['timestamp'] 

    df1['anio'] = df1['timestamp'].dt.year
    df1['mes'] = df1['timestamp'].dt.month_name(locale='es_ES')
    df1['dia'] = df1['timestamp'].dt.day
    df1['hora'] = df1['timestamp'].dt.strftime('%H:%M')

    # Crear dimensiones únicas
    dim_tiempo = df1[['fecha']].drop_duplicates().reset_index(drop=True)
    dim_tiempo['anio'] = df1['anio']
    dim_tiempo['mes'] = df1['mes']
    dim_tiempo['dia'] = df1['dia']
    dim_tiempo['hora'] = df1['hora']

    dim_tiempo['id_tiempo'] = dim_tiempo.index + 1

    dim_metodo = pd.DataFrame(df1['metodo'].unique(), columns=['metodo'])
    dim_metodo['id_metodo'] = dim_metodo.index + 1

    dim_ruta = pd.DataFrame(df1['ruta'].unique(), columns=['ruta'])
    dim_ruta['id_ruta'] = dim_ruta.index + 1

    dim_codigo_estado = pd.DataFrame(df1['codigo_estado'].unique(), columns=['codigo_estado'])
    dim_codigo_estado['id_codigo_estado'] = dim_codigo_estado.index + 1

    dim_direccion_ip = pd.DataFrame(df1['direccion_ip'].unique(), columns=['direccion_ip'])
    dim_direccion_ip['id_direccion_ip'] = dim_direccion_ip.index + 1

    dim_agente_usuario = pd.DataFrame(df1['agente_usuario'].unique(), columns=['agente_usuario'])
    dim_agente_usuario['id_agente_usuario'] = dim_agente_usuario.index + 1

    dim_referer = pd.DataFrame(df1['referer'].unique(), columns=['referer'])
    dim_referer['id_referer'] = dim_referer.index + 1
    
    # Crear tabla de hechos
    tabla_hechos = df1.merge(dim_tiempo, on=['fecha']) \
                     .merge(dim_metodo, on='metodo') \
                     .merge(dim_ruta, on='ruta') \
                     .merge(dim_codigo_estado, on='codigo_estado') \
                     .merge(dim_direccion_ip, on='direccion_ip') \
                     .merge(dim_agente_usuario, on='agente_usuario') \
                     .merge(dim_referer, on='referer')
    
    tabla_hechos = tabla_hechos[[
        'id_tiempo', 'id_metodo', 'id_ruta', 'id_codigo_estado', 
        'id_direccion_ip', 'id_agente_usuario', 'id_referer', 
        'bytes_enviados'
    ]]
    tabla_hechos['conteo_solicitudes'] = 1  # Para contar solicitudes
    
    return tabla_hechos, dim_tiempo, dim_metodo, dim_ruta, dim_codigo_estado, dim_direccion_ip, dim_agente_usuario, dim_referer


tabla_hechos, dim_tiempo, dim_metodo, dim_ruta, dim_codigo_estado, dim_direccion_ip, dim_agente_usuario, dim_referer = crear_cubo_datos(df)

# Mostrar resultados
print("Tabla de Hechos:")
print(tabla_hechos.head())
print("\nDimensión Tiempo:")
print(dim_tiempo.head())


Tabla de Hechos:
   id_tiempo  id_metodo  id_ruta  id_codigo_estado  id_direccion_ip  \
0          1          1        1                 1                1   
1          2          1        2                 2                2   
2          3          2        3                 3                3   
3          4          2        4                 3                4   
4          5          1        2                 3                5   

   id_agente_usuario  id_referer bytes_enviados  conteo_solicitudes  
0                  1           1           1391                   1  
1                  2           1            747                   1  
2                  3           2            435                   1  
3                  2           1            907                   1  
4                  2           1         792426                   1  

Dimensión Tiempo:
                fecha  anio    mes  dia   hora  id_tiempo
0 2023-01-29 03:50:28  2023  Enero   29  03:50          1
1

**INSERTAR EN CUBO MYSQL**

**INSERTAR DIM TIEMPO**

In [22]:
try:
    # Conectar a MySQL
    conexion = mysql.connector.connect(
        host='localhost',  # Cambia según tu servidor
        user='root',       # Reemplaza con tu usuario
        password='1234',   # Reemplaza con tu contraseña
        database='log_analytics'
    )
    cursor = conexion.cursor()

    # Insertar dimensiones
    for _, row in dim_tiempo.iterrows():
        cursor.execute("""
            INSERT INTO dim_tiempo (id_tiempo, anio, mes, dia, hora, fecha)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE anio=VALUES(anio), mes=VALUES(mes), dia=VALUES(dia), hora=VALUES(hora), fecha=VALUES(fecha)
        """, (row['id_tiempo'], row['anio'], row['mes'], row['dia'], row['hora'], row['fecha'].strftime('%Y-%m-%d %H:%M:%S')))

    # Confirmar los cambios
    conexion.commit()

except mysql.connector.Error as e:
    print("Error al insertar:", e)
    conexion.rollback()  # <- opcional: revierte si hubo error
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()


**INSERTAR DIM METODO**

In [6]:
try:
    # Conectar a MySQL
    conexion = mysql.connector.connect(
        host='localhost',  # Cambia según tu servidor
        user='root',       # Reemplaza con tu usuario
        password='1234',   # Reemplaza con tu contraseña
        database='log_analytics'
    )
    cursor = conexion.cursor()

    # Insertar dimensiones
    for _, row in dim_metodo.iterrows():
        cursor.execute("""
            INSERT INTO dim_metodo (id_metodo, metodo)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE metodo=VALUES(metodo)
        """, (row['id_metodo'], row['metodo']))

    # Confirmar los cambios
    conexion.commit()

except mysql.connector.Error as e:
    print("Error al insertar:", e)
    conexion.rollback()  # <- opcional: revierte si hubo error
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()


**DIMENSION RUTA**

In [7]:
try:
    # Conectar a MySQL
    conexion = mysql.connector.connect(
        host='localhost',  # Cambia según tu servidor
        user='root',       # Reemplaza con tu usuario
        password='1234',   # Reemplaza con tu contraseña
        database='log_analytics'
    )
    cursor = conexion.cursor()

    # Insertar dimensiones
    for _, row in dim_ruta.iterrows():
        cursor.execute("""
            INSERT INTO dim_ruta (id_ruta, ruta)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE ruta=VALUES(ruta)
        """, (row['id_ruta'], row['ruta']))

    # Confirmar los cambios
    conexion.commit()

except mysql.connector.Error as e:
    print("Error al insertar:", e)
    conexion.rollback()  # <- opcional: revierte si hubo error
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()


**DIMENSION CODIGO ESTADO**

In [8]:
try:
    # Conectar a MySQL
    conexion = mysql.connector.connect(
        host='localhost',  # Cambia según tu servidor
        user='root',       # Reemplaza con tu usuario
        password='1234',   # Reemplaza con tu contraseña
        database='log_analytics'
    )
    cursor = conexion.cursor()

# Insertar dimensiones
    for _, row in dim_codigo_estado.iterrows():
        cursor.execute("""
            INSERT INTO dim_codigo_estado (id_codigo_estado, codigo_estado)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE codigo_estado=VALUES(codigo_estado)
        """, (row['id_codigo_estado'], row['codigo_estado']))

    # Confirmar los cambios
    conexion.commit()

except mysql.connector.Error as e:
    print("Error al insertar:", e)
    conexion.rollback()  # <- opcional: revierte si hubo error
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()


**DIMENSION DIRRECCION IP**

In [9]:
try:
    # Conectar a MySQL
    conexion = mysql.connector.connect(
        host='localhost',  # Cambia según tu servidor
        user='root',       # Reemplaza con tu usuario
        password='1234',   # Reemplaza con tu contraseña
        database='log_analytics'
    )
    cursor = conexion.cursor()

# Insertar dimensiones
    for _, row in dim_direccion_ip.iterrows():
        cursor.execute("""
            INSERT INTO dim_direccion_ip (id_direccion_ip, direccion_ip)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE direccion_ip=VALUES(direccion_ip)
        """, (row['id_direccion_ip'], row['direccion_ip']))

    # Confirmar los cambios
    conexion.commit()

except mysql.connector.Error as e:
    print("Error al insertar:", e)
    conexion.rollback()  # <- opcional: revierte si hubo error
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()


**DIM AGENTE USUARIO**

In [10]:
try:
    # Conectar a MySQL
    conexion = mysql.connector.connect(
        host='localhost',  # Cambia según tu servidor
        user='root',       # Reemplaza con tu usuario
        password='1234',   # Reemplaza con tu contraseña
        database='log_analytics'
    )
    cursor = conexion.cursor()

# Insertar dimensiones
    for _, row in dim_agente_usuario.iterrows():
        cursor.execute("""
            INSERT INTO dim_agente_usuario (id_agente_usuario, agente_usuario)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE agente_usuario=VALUES(agente_usuario)
        """, (row['id_agente_usuario'], row['agente_usuario']))

    # Confirmar los cambios
    conexion.commit()

except mysql.connector.Error as e:
    print("Error al insertar:", e)
    conexion.rollback()  # <- opcional: revierte si hubo error
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()


**DIMENSION REFER**

In [13]:
try:
    # Conectar a MySQL
    conexion = mysql.connector.connect(
        host='localhost',  # Cambia según tu servidor
        user='root',       # Reemplaza con tu usuario
        password='1234',   # Reemplaza con tu contraseña
        database='log_analytics'
    )
    cursor = conexion.cursor()

# Insertar dimensiones
    for _, row in dim_referer.iterrows():
        cursor.execute("""
            INSERT INTO dim_referer (id_referer, referer)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE referer=VALUES(referer)
        """, (row['id_referer'], row['referer']))

    # Confirmar los cambios
    conexion.commit()

except mysql.connector.Error as e:
    print("Error al insertar:", e)
    conexion.rollback()  # <- opcional: revierte si hubo error
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()


**TABLA HECHO**

In [23]:
try:
    # Conectar a MySQL
    conexion = mysql.connector.connect(
        host='localhost',  # Cambia según tu servidor
        user='root',       # Reemplaza con tu usuario
        password='1234',   # Reemplaza con tu contraseña
        database='log_analytics'
    )
    cursor = conexion.cursor()

# Insertar dimensiones
    for _, row in tabla_hechos.iterrows():
        cursor.execute("""
            INSERT INTO tabla_hechos (
                id_tiempo, id_metodo, id_ruta, id_codigo_estado, 
                id_direccion_ip, id_agente_usuario, id_referer, 
                bytes_enviados, conteo_solicitudes
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['id_tiempo'], row['id_metodo'], row['id_ruta'], row['id_codigo_estado'],
            row['id_direccion_ip'], row['id_agente_usuario'], row['id_referer'],
            row['bytes_enviados'], row['conteo_solicitudes']
        ))

    # Confirmar los cambios
    conexion.commit()

except mysql.connector.Error as e:
    print("Error al insertar:", e)
    conexion.rollback()  # <- opcional: revierte si hubo error
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()


In [4]:
# Función para insertar datos en MySQL
def insertar_datos_mysql(tabla_hechos, dim_tiempo, dim_metodo, dim_ruta, dim_codigo_estado, dim_direccion_ip, dim_agente_usuario, dim_referer):
    try:
        # Conectar a MySQL
        conexion = mysql.connector.connect(
            host='localhost',  # Cambia según tu servidor
            user='root',  # Reemplaza con tu usuario
            password='1234',  # Reemplaza con tu contraseña
            database='log_analytics'
        )
        cursor = conexion.cursor()

        # Insertar dimensiones
        # dim_tiempo
        for _, row in dim_tiempo.iterrows():
            cursor.execute("""
                INSERT INTO dim_tiempo (id_tiempo, anio, mes, dia, hora)
                VALUES (%s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE anio=VALUES(anio), mes=VALUES(mes), dia=VALUES(dia), hora=VALUES(hora)
            """, (row['id_tiempo'], row['anio'], row['mes'], row['dia'], row['hora']))

        # dim_metodo
        for _, row in dim_metodo.iterrows():
            cursor.execute("""
                INSERT INTO dim_metodo (id_metodo, metodo)
                VALUES (%s, %s)
                ON DUPLICATE KEY UPDATE metodo=VALUES(metodo)
            """, (row['id_metodo'], row['metodo']))

        # dim_ruta
        for _, row in dim_ruta.iterrows():
            cursor.execute("""
                INSERT INTO dim_ruta (id_ruta, ruta)
                VALUES (%s, %s)
                ON DUPLICATE KEY UPDATE ruta=VALUES(ruta)
            """, (row['id_ruta'], row['ruta']))

        # dim_codigo_estado
        for _, row in dim_codigo_estado.iterrows():
            cursor.execute("""
                INSERT INTO dim_codigo_estado (id_codigo_estado, codigo_estado)
                VALUES (%s, %s)
                ON DUPLICATE KEY UPDATE codigo_estado=VALUES(codigo_estado)
            """, (row['id_codigo_estado'], row['codigo_estado']))

        # dim_direccion_ip
        for _, row in dim_direccion_ip.iterrows():
            cursor.execute("""
                INSERT INTO dim_direccion_ip (id_direccion_ip, direccion_ip)
                VALUES (%s, %s)
                ON DUPLICATE KEY UPDATE direccion_ip=VALUES(direccion_ip)
            """, (row['id_direccion_ip'], row['direccion_ip']))

        # dim_agente_usuario
        for _, row in dim_agente_usuario.iterrows():
            cursor.execute("""
                INSERT INTO dim_agente_usuario (id_agente_usuario, agente_usuario)
                VALUES (%s, %s)
                ON DUPLICATE KEY UPDATE agente_usuario=VALUES(agente_usuario)
            """, (row['id_agente_usuario'], row['agente_usuario']))

        # dim_referer
        for _, row in dim_referer.iterrows():
            cursor.execute("""
                INSERT INTO dim_referer (id_referer, referer)
                VALUES (%s, %s)
                ON DUPLICATE KEY UPDATE referer=VALUES(referer)
            """, (row['id_referer'], row['referer']))

        # Insertar tabla de hechos
        for _, row in tabla_hechos.iterrows():
            cursor.execute("""
                INSERT INTO tabla_hechos (
                    id_tiempo, id_metodo, id_ruta, id_codigo_estado, 
                    id_direccion_ip, id_agente_usuario, id_referer, 
                    bytes_enviados, conteo_solicitudes
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                row['id_tiempo'], row['id_metodo'], row['id_ruta'], row['id_codigo_estado'],
                row['id_direccion_ip'], row['id_agente_usuario'], row['id_referer'],
                row['bytes_enviados'], row['conteo_solicitudes']
            ))

        # Confirmar los cambios
        conexion.commit()
        print("Datos insertados correctamente en MySQL.")

    except Error as e:
        print(f"Error al conectar o insertar en MySQL: {e}")
    finally:
        if conexion.is_connected():
            cursor.close()
            conexion.close()
            print("Conexión a MySQL cerrada.")

insertar_datos_mysql(tabla_hechos, dim_tiempo, dim_metodo, dim_ruta, dim_codigo_estado, dim_direccion_ip, dim_agente_usuario, dim_referer)

Error al conectar o insertar en MySQL: 1406 (22001): Data too long for column 'referer' at row 1
Conexión a MySQL cerrada.
